In [ ]:
#@title Imports
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.colheader_justify', 'center')
import numpy as np
import random
import time
import re

import scipy.stats
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

from inspect import signature

def someMethod(self, arg1, kwarg1=None):
    pass

nba_possession_data_seasons = np.arange(1974, 2024, 1)
play_by_play_data_seasons = np.arange(1997, 2024, 1)
nba_pre_possession_data_seasons = np.arange(1952, 1974, 1)
aba_possession_data_seasons = np.arange(1974, 1977, 1)
aba_pre_possession_data_seasons = np.arange(1968, 1974, 1)

all_nba_seasons = np.arange(1952, 2024, 1)
all_aba_seasons = np.arange(1968, 1977, 1)

# plot
import seaborn as sns
import ast
from matplotlib.lines import Line2D
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
sns.set_style('darkgrid')        # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:
      ax.text(point['x']+0.12, point['y']+.25, str(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))

def label_point_year(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:
      ax.text(point['x']+0.12, point['y'], int(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))

plot_colors_set_list = ["#FF0000", "#D3A6D6", "#916613", "#00A4A0", "#FF7F00", "#ABFFD2", "#610077", "#173E4C", "#00A54C", "#FF00A1", "#FFFE00", "#0B1ADD"]

In [ ]:
#@title Import Selenium
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
#@title Start Webdriver and VirtualDisplay
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(service=service, options=options)

#options = Options()
#options.add_argument("--headless")
#options.add_argument("--no-sandbox")
#options.headless = True

#wd = webdriver.Chrome("/usr/bin/chromedriver", options=options)

!pip install pyvirtualdisplay
!apt-get install xvfb
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))
display.start()

In [ ]:
# @title Determine Opponents' DefRtg+
teams = pd.read_csv('/content/Nash_Series.csv', index_col=False, encoding='utf8')

specific_team = "LEBRON"

specific_team_series = teams[(teams['Team'] == specific_team)]
specific_team_series = specific_team_series[(specific_team_series['Year'] >= 2008)]
specific_team_series = specific_team_series[(specific_team_series['Year'] <= 2018)]

total_defrtg = 0
stat = 'DefRtg+'

total_opponents_df = pd.DataFrame(columns=[f'{stat}', 'MP'])

team_finished = []

for idx, row in specific_team_series.iterrows():

  opp = row['Opp']
  year = row['Year']
  opp_playoffs = teams[(teams['Team'] == opp) & (teams['Year'] == year)]
  opp_playoffs = opp_playoffs[opp_playoffs['Opp'] != specific_team]

  if not opp_playoffs.empty:
    team_was = str(row['Opp']) + str(row['Year'])
    if team_was not in team_finished:
      mp = int(opp_playoffs['MP'].sum())
      opp_playoffs[f'{stat}_Portion'] = opp_playoffs[f'{stat}'] * (opp_playoffs['MP'] / mp)
      rtg = opp_playoffs[f'{stat}_Portion'].sum().round(1)
      total_opponents_df.loc[len(total_opponents_df.index)] = [rtg, mp]
      print(opp_playoffs)
      team_finished.append(team_was)


mp = int(total_opponents_df['MP'].sum())
total_opponents_df['Portion'] = total_opponents_df[f'{stat}'] = total_opponents_df[f'{stat}'] * (total_opponents_df['MP'] / mp)
rtg = total_opponents_df[f'Portion'].sum().round(1)
print(rtg)
print(mp)

In [ ]:
# @title Determine Opponents' OffRtg+
teams = pd.read_csv('/content/nba_Playoff_rOffRtg_rDefRtg_Series_By_Series_1957_2023_df.csv', index_col=False, encoding='utf8')

specific_team = "SAS"
min_year = 2001
max_year = 2008

specific_team_series = teams[(teams['Team'] == specific_team)]
specific_team_series = specific_team_series[(specific_team_series['Year'] >= min_year)]
specific_team_series = specific_team_series[(specific_team_series['Year'] <= max_year)]

total_defrtg = 0
stat = 'OffRtg+'

total_opponents_df = pd.DataFrame(columns=[f'{stat}', 'MP'])

team_finished = []

for idx, row in specific_team_series.iterrows():

  opp = row['Opp']
  year = row['Year']
  opp_playoffs = teams[(teams['Team'] == opp) & (teams['Year'] == year)]
  opp_playoffs = opp_playoffs[opp_playoffs['Opp'] != specific_team]

  if not opp_playoffs.empty:
    team_was = str(row['Opp']) + str(row['Year'])
    if team_was not in team_finished:
      mp = int(opp_playoffs['MP'].sum())
      opp_playoffs[f'{stat}_Portion'] = opp_playoffs[f'{stat}'] * (opp_playoffs['MP'] / mp)
      rtg = opp_playoffs[f'{stat}_Portion'].sum().round(1)
      total_opponents_df.loc[len(total_opponents_df.index)] = [rtg, mp]
      print(opp_playoffs)
      tmp_sum = opp_playoffs[f'{stat}_Portion'].sum().round(1)
      team_finished.append(team_was)
      print(tmp_sum)


mp = int(total_opponents_df['MP'].sum())
total_opponents_df['Portion'] = total_opponents_df[f'{stat}'] = total_opponents_df[f'{stat}'] * (total_opponents_df['MP'] / mp)
rtg = total_opponents_df[f'Portion'].sum().round(1)
print(rtg)
print(mp)

In [ ]:
# @title Print OffRtg or DefRtg over a stretch for a team
def print_rtg_over_stretch(stat, team, first_year, second_year):
  teams = pd.read_csv('/content/nba_Playoff_rOffRtg_rDefRtg_1957_2023_df.csv', index_col=False, encoding='utf8')

  if stat == "d":
    stat = "DefRtg+"
  elif stat == "o":
    stat = "OffRtg+"
  elif stat == "n":
    stat = "NetRtg+"
  specific_team = teams[(teams['Year'] >= first_year)]
  specific_team = specific_team[(specific_team['Year'] <= second_year)]
  specific_team = specific_team[(specific_team['Team'] == team)]
  mp = specific_team['MP'].sum()
  print("MP: ", mp)

  specific_team[f'{stat}_Portion'] = specific_team[f'{stat}'] * (specific_team['MP'] / mp)
  peak = specific_team[f'{stat}_Portion'].sum().round(1)

  print(f"{stat}: ", peak)

In [ ]:
print_rtg_over_stretch("d", "DET", 2000, 2003)

MP:  1450
DefRtg+:  103.4


In [ ]:
# @title Scrape Playoff Teams' OffRtg+ and rDefRtg+
def scrape_playoff_rOffRtg_rDefRtg(origal_team_df):


  teams_defense = pd.read_csv('/content/nba_Team_DefRtg_Allowed_74-23_df.csv', index_col=False, encoding='utf8')
  teams_offense = pd.read_csv('/content/nba_Team_OffRtg_74-23_df.csv', index_col=False, encoding='utf8')

  fiftytwo_seventythree_off_def_rtgs = pd.read_csv('/content/nba_Team_Estimated_Pace_52-73_df.csv', index_col=False, encoding='utf8')

  missing_pistons_lakers_60_62 = pd.read_csv('/content/missing_fga_60_62.csv', index_col=False, encoding='utf8')


  team_df = pd.DataFrame(columns = ['Year', 'Team', 'OffRtg+', 'DefRtg+', 'MP'])
  total_series_df = pd.DataFrame(columns = ['Year', 'Team', 'Opp', 'OffRtg+', 'DefRtg+', 'MP'])

  for idx, row in origal_team_df.iterrows():

    use_different_pace_formula = 0
    dont_estimate_pace = 0

    original_team = str(row['Team'])
    series_df = pd.DataFrame(columns = ['Year', 'Team', 'Opp', 'OffRtg+', 'DefRtg+', 'MP'])

    team_url = "https://www.basketball-reference.com/teams/" + row['Team'] + "/" + str(row['Year']) + ".html"

    html = urlopen(team_url)
    soup = BeautifulSoup(html, features="lxml")

    for first_div in soup.find_all('div', attrs={'id': 'info'}):
      second_div = first_div.find('div', attrs={'id': 'meta'})
      second_div = str(second_div)
      ref_urls = re.findall(r'/\w+\/\d+[0-9abcdefghijklmnopqrstuvwxyz-]+', second_div)
      urls = []
      for ref_url in ref_urls:
        if "eastern" in ref_url or "western" in ref_url or "finals" in ref_url:
          ref_url = "https://www.basketball-reference.com" + ref_url + ".html"
          urls.append(ref_url)
    for series_url in urls:

      time.sleep(6)
      html = urlopen(series_url)
      soup = BeautifulSoup(html, features="lxml")

      for first_div in soup.find_all('div', attrs={'id': 'content'}):
        second_div = first_div.find('div', attrs={'id': 'all_four_factors'})

      second_div = str(second_div)

      teams = re.findall(r'/\w+\/\d+', second_div)
      count = 0
      for team in teams:
        if count == 0:
          if str(row['Team']) in team:
            team_in = 0
          else:
            team_in = 1
            other_team = team
        else:
          if str(row['Team']) not in team:
            other_team = team
        count = count + 1
      year = row['Year']
      other_team = other_team.replace(f'{year}', '')
      other_team = other_team.replace(f'/', '')

      rtgs_untrimmed = re.findall(r'off_rtg" >\d+.\d+', second_div)

      # no OffRtg's. Need to estimate them given boxscore.
      if not rtgs_untrimmed:
        if year >= 1971:
          ORB_percent = 0.319
        else:
          ORB_percent = 0.303

        # most likely have TOV data
        if year >= 1974:

          # team data to estimate pace

          first_div = soup.find('div', attrs={'id': 'content'})
          second_div = first_div.find('div', attrs={'id': f'all_{original_team}'})
          first_table = second_div.find('table', attrs={'id': f'{original_team}'})
          if first_table == None:
            second_div_str = str(second_div)
            mp_untrimmed = re.findall(r"data-stat=[\"]mp\" >\d+", second_div_str)
            mp_untrimmed = mp_untrimmed[-1]
            mp = mp_untrimmed.replace(f'data-stat="mp" >', '')
            mp = int(mp) // 5

            fga_untrimmed = re.findall(r"data-stat=[\"]fga\" >\d+", second_div_str)
            fga_untrimmed = fga_untrimmed[-1]
            fga = fga_untrimmed.replace(f'data-stat="fga" >', '')

            fg_untrimmed = re.findall(r"data-stat=[\"]fg\" >\d+", second_div_str)
            fg_untrimmed = fg_untrimmed[-1]
            fg = fg_untrimmed.replace(f'data-stat="fg" >', '')

            fta_untrimmed = re.findall(r"data-stat=[\"]fta\" >\d+", second_div_str)
            fta_untrimmed = fta_untrimmed[-1]
            fta = fta_untrimmed.replace(f'data-stat="fta" >', '')
            ft_trips = fta * .44

            tov_untrimmed = re.findall(r"data-stat=[\"]tov\" >\d+", second_div_str)
            tov_untrimmed = tov_untrimmed[-1]

            pts_untrimmed = re.findall(r"data-stat=[\"]pts\" >\d+", second_div_str)
            pts_untrimmed = pts_untrimmed[-1]
            pts = pts_untrimmed.replace(f'data-stat="pts" >', '')

            if tov:
              orb_untrimmed = re.findall(r"data-stat=[\"]orb\" >\d+", second_div_str)
              if orb_untrimmed:
                orb_untrimmed = orb_untrimmed[-1]
                orb = orb_untrimmed.replace(f'data-stat="orb" >', '')
                poss = fga + ft_trips + tov - orb
                dont_estimate_pace = 1
              else:
                poss = fga + ft_trips + tov - (ORB_percent * (fga - fg))
              TOV_percent = tov / poss
            else:
              orb_untrimmed = re.findall(r"data-stat=[\"]orb\" >\d+", second_div_str)
              if orb_untrimmed:
                orb_untrimmed = orb_untrimmed[-1]
                orb = orb_untrimmed.replace(f'data-stat="orb" >', '')
                use_different_pace_formula = 1
              else:
                TOV_percent = 0.158

          else:
            header = first_table.find('thead')
            foot = first_table.find('tfoot')

            headers = [th.getText() for th in header.findAll('tr', limit=2)[1].findAll('th')]
            rows = foot.findAll('tr')[0:]
            series_stats = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

            # remove empty rows
            series_stats = [e for e in series_stats if e != []]
            series_stats = ([i for i in series_stats if i[0] != 'Year'])

            #create df
            series_data = pd.DataFrame(series_stats, columns = headers)
            pts_count = 0
            mp_count = 0
            cols = []
            for column in series_data.columns:
              if column == 'MP':
                if mp_count == 1:
                  cols.append(f'MPG')
                else:
                  cols.append(f'MP')
                mp_count+=1
                continue
              if column == 'PTS':
                if pts_count == 1:
                  cols.append(f'PTS/G')
                else:
                  cols.append(f'PTS')
                pts_count+=1
                continue
              cols.append(column)
            series_data.columns = cols

            mp = int(series_data['MP'])
            mp = int(mp // 5)

            fga = int(series_data['FGA'])

            fg = int(series_data['FG'])

            fta = int(series_data['FTA'])
            ft_trips = fta * .44

            pts = int(series_data['PTS'])

            try:
              tov = int(series_data['TOV'])
              try:
                orb = int(series_data['ORB'])
                poss = fga + ft_trips + tov - orb
                dont_estimate_pace = 1
              except:
                poss = fga + ft_trips + tov - (ORB_percent * (fga - fg))
                TOV_percent = tov / poss
            except:
              try:
                orb = int(series_data['ORB'])
                use_different_pace_formula = 1
              except:
                TOV_percent = 0.158

          if dont_estimate_pace == 1:
            OffRtg = pts / poss * 100
          elif use_different_pace_formula == 0:
            original_team_possessions = fga + (0.4 * fta) - ORB_percent * (fga - fg) + (-TOV_percent * (fga + 0.44 * fta) / (TOV_percent - 1))
            OffRtg = pts / original_team_possessions * 100
          else:
            original_team_possessions = fga + (0.4 * fta) - orb + (-TOV_percent * (fga + 0.44 * fta) / (TOV_percent - 1))
            OffRtg = pts / original_team_possessions * 100

          # opponent data to estimate opponent pace

          use_different_pace_formula = 0
          dont_estimate_pace = 0

          first_div = soup.find('div', attrs={'id': 'content'})
          second_div = first_div.find('div', attrs={'id': f'all_{other_team}'})
          first_table = second_div.find('table', attrs={'id': f'{other_team}'})
          if first_table == None:
            second_div_str = str(second_div)

            fga_untrimmed = re.findall(r"data-stat=[\"]fga\" >\d+", second_div_str)
            fga_untrimmed = fga_untrimmed[-1]
            fga = fga_untrimmed.replace(f'data-stat="fga" >', '')

            fg_untrimmed = re.findall(r"data-stat=[\"]fg\" >\d+", second_div_str)
            fg_untrimmed = fg_untrimmed[-1]
            fg = fg_untrimmed.replace(f'data-stat="fg" >', '')

            fta_untrimmed = re.findall(r"data-stat=[\"]fta\" >\d+", second_div_str)
            fta_untrimmed = fta_untrimmed[-1]
            fta = fta_untrimmed.replace(f'data-stat="fta" >', '')
            ft_trips = fta * .44

            tov_untrimmed = re.findall(r"data-stat=[\"]tov\" >\d+", second_div_str)
            tov_untrimmed = tov_untrimmed[-1]
            tov = tov_untrimmed.replace(f'data-stat="tov" >', '')

            pts_untrimmed = re.findall(r"data-stat=[\"]pts\" >\d+", second_div_str)
            pts_untrimmed = pts_untrimmed[-1]
            pts = pts_untrimmed.replace(f'data-stat="pts" >', '')

            if tov:
              orb_untrimmed = re.findall(r"data-stat=[\"]orb\" >\d+", second_div_str)
              if orb_untrimmed:
                orb_untrimmed = orb_untrimmed[-1]
                orb = orb_untrimmed.replace(f'data-stat="orb" >', '')
                poss = fga + ft_trips + tov - orb
                dont_estimate_pace = 1
              else:
                poss = fga + ft_trips + tov - (ORB_percent * (fga - fg))
              TOV_percent = tov / poss
            else:
              orb_untrimmed = re.findall(r"data-stat=[\"]orb\" >\d+", second_div_str)
              if orb_untrimmed:
                orb_untrimmed = orb_untrimmed[-1]
                orb = orb_untrimmed.replace(f'data-stat="orb" >', '')
                use_different_pace_formula = 1
              else:
                  TOV_percent = 0.158

          else:
            header = first_table.find('thead')
            foot = first_table.find('tfoot')

            headers = [th.getText() for th in header.findAll('tr', limit=2)[1].findAll('th')]
            rows = foot.findAll('tr')[0:]
            series_stats = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

            # remove empty rows
            series_stats = [e for e in series_stats if e != []]
            series_stats = ([i for i in series_stats if i[0] != 'Year'])

            #create df
            series_data = pd.DataFrame(series_stats, columns = headers)
            pts_count = 0
            mp_count = 0
            cols = []
            for column in series_data.columns:
              if column == 'MP':
                if mp_count == 1:
                  cols.append(f'MPG')
                else:
                  cols.append(f'MP')
                mp_count+=1
                continue
              if column == 'PTS':
                if pts_count == 1:
                  cols.append(f'PTS/G')
                else:
                  cols.append(f'PTS')
                pts_count+=1
                continue
              cols.append(column)
            series_data.columns = cols

            fga = int(series_data['FGA'])

            fg = int(series_data['FG'])

            fta = int(series_data['FTA'])
            ft_trips = fta * .44

            pts = int(series_data['PTS'])

            try:
              tov = int(series_data['TOV'])
              try:
                orb = int(series_data['ORB'])
                poss = fga + ft_trips + tov - orb
                dont_estimate_pace = 1
              except:
                poss = fga + ft_trips + tov - (ORB_percent * (fga - fg))
                TOV_percent = tov / poss
            except:
              try:
                orb = int(series_data['ORB'])
                use_different_pace_formula = 1
              except:
                TOV_percent = 0.158

          if dont_estimate_pace == 1:
            DefRtg = pts / poss * 100
          elif use_different_pace_formula == 0:
            opponent_team_possessions = fga + (0.4 * fta) - ORB_percent * (fga - fg) + (-TOV_percent * (fga + 0.44 * fta) / (TOV_percent - 1))
            DefRtg = pts / opponent_team_possessions * 100
          else:
            opponent_team_possessions = fga + (0.4 * fta) - orb + (-TOV_percent * (fga + 0.44 * fta) / (TOV_percent - 1))
            DefRtg = pts / opponent_team_possessions * 100

          other_team_defrtg = teams_defense[(teams_defense['Year'] == year) & (teams_defense['Team'] == other_team)]
          other_team_offrtg = teams_offense[(teams_offense['Year'] == year) & (teams_offense['Team'] == other_team)]

          r_OffRtg = float(OffRtg) / float(other_team_defrtg['DefRtg']) * 100
          r_DefRtg = float(other_team_offrtg['OffRtg']) / float(DefRtg) * 100

          r_OffRtg = round(r_OffRtg, 1)
          r_DefRtg = round(r_DefRtg, 1)

        # will not have TOV data
        else:
          if year >= 1971:
            TOV_percent = 0.158
          else:
            TOV_percent = 0.161
          first_div = soup.find('div', attrs={'id': 'content'})
          second_div = first_div.find('div', attrs={'id': f'all_{original_team}'})
          first_table = second_div.find('table', attrs={'id': f'{original_team}'})
          if first_table == None:
            second_div_str = str(second_div)
            mp_untrimmed = re.findall(r"data-stat=[\"]mp\" >\d+", second_div_str)
            mp_untrimmed = mp_untrimmed[-1]
            mp = mp_untrimmed.replace(f'data-stat="mp" >', '')
            mp = int(mp) // 5

            fga_untrimmed = re.findall(r"data-stat=[\"]fga\" >\d+", second_div_str)
            fga_untrimmed = fga_untrimmed[-1]
            fga = fga_untrimmed.replace(f'data-stat="fga" >', '')

            fg_untrimmed = re.findall(r"data-stat=[\"]fg\" >\d+", second_div_str)
            fg_untrimmed = fg_untrimmed[-1]
            fg = fg_untrimmed.replace(f'data-stat="fg" >', '')

            fta_untrimmed = re.findall(r"data-stat=[\"]fta\" >\d+", second_div_str)
            fta_untrimmed = fta_untrimmed[-1]
            fta = fta_untrimmed.replace(f'data-stat="fta" >', '')

            pts_untrimmed = re.findall(r"data-stat=[\"]pts\" >\d+", second_div_str)
            pts_untrimmed = pts_untrimmed[-1]
            pts = pts_untrimmed.replace(f'data-stat="pts" >', '')

            fta = int(fta)
            fga = int(fga)
            fg = int(fg)
            pts = int(pts)


            total_fga = fga
            total_fg = fg
            total_fta = fta
            total_pts = pts

            original_pts = pts

          else:
            header = first_table.find('thead')
            foot = first_table.find('tfoot')

            headers = [th.getText() for th in header.findAll('tr', limit=2)[1].findAll('th')]
            rows = foot.findAll('tr')[0:]
            series_stats = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

            # remove empty rows
            series_stats = [e for e in series_stats if e != []]
            series_stats = ([i for i in series_stats if i[0] != 'Year'])

            #create df
            series_data = pd.DataFrame(series_stats, columns = headers)
            pts_count = 0
            mp_count = 0
            cols = []
            for column in series_data.columns:
              if column == 'MP':
                if mp_count == 1:
                  cols.append(f'MPG')
                else:
                  cols.append(f'MP')
                mp_count+=1
                continue
              if column == 'PTS':
                if pts_count == 1:
                  cols.append(f'PTS/G')
                else:
                  cols.append(f'PTS')
                pts_count+=1
                continue
              cols.append(column)
            series_data.columns = cols

            mp = int(series_data['MP'])
            mp = int(mp // 5)

            try:
              fga = int(series_data['FGA'])
            except:
              fga = missing_pistons_lakers_60_62[(missing_pistons_lakers_60_62['Team'] == original_team) & (missing_pistons_lakers_60_62['Year'] == year)]
              fga = fga['FGA']
              fga = int(fga)

            fg = int(series_data['FG'])

            fta = int(series_data['FTA'])
            ft_trips = fta * .44

            pts = int(series_data['PTS'])

            fta = int(fta)
            fga = int(fga)
            fg = int(fg)

            total_fga = fga
            total_fg = fg
            total_fta = fta
            total_pts = pts

            original_pts = pts

          # opponent data to estimate opponent pace

          first_div = soup.find('div', attrs={'id': 'content'})
          second_div = first_div.find('div', attrs={'id': f'all_{other_team}'})
          first_table = second_div.find('table', attrs={'id': f'{other_team}'})
          if first_table == None:
            second_div_str = str(second_div)

            fga_untrimmed = re.findall(r"data-stat=[\"]fga\" >\d+", second_div_str)
            fga_untrimmed = fga_untrimmed[-1]
            fga = fga_untrimmed.replace(f'data-stat="fga" >', '')

            fg_untrimmed = re.findall(r"data-stat=[\"]fg\" >\d+", second_div_str)
            fg_untrimmed = fg_untrimmed[-1]
            fg = fg_untrimmed.replace(f'data-stat="fg" >', '')

            fta_untrimmed = re.findall(r"data-stat=[\"]fta\" >\d+", second_div_str)
            fta_untrimmed = fta_untrimmed[-1]
            fta = fta_untrimmed.replace(f'data-stat="fta" >', '')

            pts_untrimmed = re.findall(r"data-stat=[\"]pts\" >\d+", second_div_str)
            pts_untrimmed = pts_untrimmed[-1]
            pts = pts_untrimmed.replace(f'data-stat="pts" >', '')

            fta = int(fta)
            fga = int(fga)
            fg = int(fg)
            pts = int(pts)

            total_fga = total_fga + fga
            total_fg = total_fg + fg
            total_fta = total_fta + fta
            total_pts = total_pts + pts

          else:
            header = first_table.find('thead')
            foot = first_table.find('tfoot')

            headers = [th.getText() for th in header.findAll('tr', limit=2)[1].findAll('th')]
            rows = foot.findAll('tr')[0:]
            series_stats = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

            # remove empty rows
            series_stats = [e for e in series_stats if e != []]
            series_stats = ([i for i in series_stats if i[0] != 'Year'])

            #create df
            series_data = pd.DataFrame(series_stats, columns = headers)
            pts_count = 0
            mp_count = 0
            cols = []
            for column in series_data.columns:
              if column == 'MP':
                if mp_count == 1:
                  cols.append(f'MPG')
                else:
                  cols.append(f'MP')
                mp_count+=1
                continue
              if column == 'PTS':
                if pts_count == 1:
                  cols.append(f'PTS/G')
                else:
                  cols.append(f'PTS')
                pts_count+=1
                continue
              cols.append(column)
            series_data.columns = cols

            try:
              fga = int(series_data['FGA'])
            except:
              fga = missing_pistons_lakers_60_62[(missing_pistons_lakers_60_62['Team'] == original_team) & (missing_pistons_lakers_60_62['Year'] == year)]
              fga = fga['FGA']
              fga = int(fga)

            fg = int(series_data['FG'])

            fta = int(series_data['FTA'])
            ft_trips = fta * .44

            pts = int(series_data['PTS'])

            fta = int(fta)
            fga = int(fga)
            fg = int(fg)

            total_fga = total_fga + fga
            total_fg = total_fg + fg
            total_fta = total_fta + fta
            total_pts = total_pts + pts

          possessions = (total_fga + (0.4 * total_fta) - ORB_percent * (total_fga - total_fg) + (-TOV_percent * (total_fga + 0.44 * total_fta) / (TOV_percent - 1))) / 2
          DefRtg = pts / possessions * 100
          OffRtg = original_pts / possessions * 100

          other_team_defrtg_ortg = fiftytwo_seventythree_off_def_rtgs[(fiftytwo_seventythree_off_def_rtgs['Year'] == year) & (fiftytwo_seventythree_off_def_rtgs['Team'] == other_team)]
          r_OffRtg = float(OffRtg) / float(other_team_defrtg_ortg['DefRtg']) * 100
          r_DefRtg = float(other_team_defrtg_ortg['OffRtg']) / float(DefRtg) * 100

          r_OffRtg = round(r_OffRtg, 1)
          r_DefRtg = round(r_DefRtg, 1)

      # have OffRtg's, can use them
      else:
        count = 0
        for rtg in rtgs_untrimmed:
          rtg = rtg.replace('off_rtg" >', '')
          if count == 0 and team_in == 0:
            off_rtg = rtg
          elif count == 1 and team_in == 0:
            def_rtg = rtg
          elif count == 0 and team_in == 1:
            def_rtg = rtg
          elif count == 1 and team_in == 1:
            off_rtg = rtg
          count = count + 1

        other_team_defrtg = teams_defense[(teams_defense['Year'] == year) & (teams_defense['Team'] == other_team)]

        other_team_offrtg = teams_offense[(teams_offense['Year'] == year) & (teams_offense['Team'] == other_team)]

        r_OffRtg = float(off_rtg) / float(other_team_defrtg['DefRtg']) * 100
        r_DefRtg = float(other_team_offrtg['OffRtg']) / float(def_rtg)* 100

        r_OffRtg = round(r_OffRtg, 1)
        r_DefRtg = round(r_DefRtg, 1)


        first_div = soup.find('div', attrs={'id': 'content'})
        second_div = first_div.find('div', attrs={'id': f'all_{original_team}'})
        first_table = second_div.find('table', attrs={'id': f'{original_team}'})
        if first_table == None:
          second_div_str = str(second_div)
          mp_untrimmed = re.findall(r"data-stat=[\"]mp\" >\d+", second_div_str)
          mp_untrimmed = mp_untrimmed[-1]
          mp = mp_untrimmed.replace(f'data-stat="mp" >', '')
          mp = int(mp) // 5
        else:
          header = first_table.find('thead')
          foot = first_table.find('tfoot')

          headers = [th.getText() for th in header.findAll('tr', limit=2)[1].findAll('th')]
          rows = foot.findAll('tr')[0:]
          series_stats = [[td.getText() for td in rows[i].findAll(['td','th'])] for i in range(len(rows))]

          # remove empty rows
          series_stats = [e for e in series_stats if e != []]
          series_stats = ([i for i in series_stats if i[0] != 'Year'])

          #create df
          series_data = pd.DataFrame(series_stats, columns = headers)
          pts_count = 0
          mp_count = 0
          cols = []
          for column in series_data.columns:
            if column == 'MP':
              if mp_count == 1:
                cols.append(f'MPG')
              else:
                cols.append(f'MP')
              mp_count+=1
              continue
            if column == 'PTS':
              if pts_count == 1:
                cols.append(f'PTS/G')
              else:
                cols.append(f'PTS')
              pts_count+=1
              continue
            cols.append(column)
          series_data.columns = cols

          mp = int(series_data['MP'])
          mp = int(mp // 5)

      new_row = pd.DataFrame(np.array([[year, original_team, other_team, r_OffRtg, r_DefRtg, mp]]), columns=['Year', 'Team', 'Opp', 'OffRtg+', 'DefRtg+', 'MP'])
      series_df = pd.concat([series_df, new_row], ignore_index=True)
      total_series_df = pd.concat([total_series_df, new_row], ignore_index=True)
      series_df['MP'] = series_df['MP'].astype(int)
      total_series_df['MP'] = total_series_df['MP'].astype(int)

    mp = int(series_df['MP'].sum())
    print(series_df)

    for idx, row in series_df.iterrows():

      series_df.loc[idx, 'OffRtg+_Portion'] = float(series_df.loc[idx, 'OffRtg+']) * float((series_df.loc[idx, 'MP'] / mp))
      series_df.loc[idx, 'DefRtg+_Portion'] = float(series_df.loc[idx, 'DefRtg+']) * float((series_df.loc[idx, 'MP'] / mp))
    rOffRtg_avg = series_df[f'OffRtg+_Portion'].sum().round(1)
    rDefRtg_avg = series_df[f'DefRtg+_Portion'].sum().round(1)

    new_row = pd.DataFrame(np.array([[year, original_team, rOffRtg_avg, rDefRtg_avg, mp]]), columns=['Year', 'Team', 'OffRtg+', 'DefRtg+', 'MP'])
    team_df = pd.concat([team_df, new_row], ignore_index=True)

    total_series_df.to_csv('nba_Playoff_rOffRtg_rDefRtg_Series_By_Series_1957_2023_df.csv', index=False)
    team_df.to_csv('nba_Playoff_rOffRtg_rDefRtg_1957_2023_df.csv', index=False)
    print(team_df)

In [ ]:
teams = pd.read_csv('/content/nba_playoff_teams_1952_2023_df.csv', index_col=False, encoding='utf8')
scrape_playoff_rOffRtg_rDefRtg(teams)

In [ ]:
teams = pd.read_csv('/content/nba_Playoff_rOffRtg_rDefRtg_Series_By_Series_1957_2023_df.csv', index_col=False, encoding='utf8')
finished = []
count = 0
for idx, row in teams.iterrows():
  if row['Team'] not in finished:
    print(row['Team'])
    specific_team = teams[(teams['Team'] == row['Team'])]
    specific_team.to_csv(f"{row['Team']}_Adjusted_Playoff_NetRtg_Series_By_Series.csv", index=False)
    finished.append(row['Team'])

In [ ]:
# @title 1000-5000 MP OffRtg+ Peaks
team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team = "LEBRON"

team_data = team_data[team_data['MP'] >= 1000]
team_data = team_data[team_data['MP'] <= 1999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)
team_data.to_csv('1000_MP_Team_Peaks.csv',index=False)
print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 2000]
team_data = team_data[team_data['MP'] <= 2999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)
team_data.to_csv('2000_MP_Team_Peaks.csv',index=False)
print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 3000]
team_data = team_data[team_data['MP'] <= 3999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)
team_data.to_csv('3000_MP_Team_Peaks.csv',index=False)
print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 4000]
team_data = team_data[team_data['MP'] <= 4999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)
team_data.to_csv('4000_MP_Team_Peaks.csv',index=False)
print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 5000]
team_data = team_data[team_data['MP'] <= 5999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('5000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 6000]
team_data = team_data[team_data['MP'] <= 6999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('5000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 7000]
team_data = team_data[team_data['MP'] <= 7999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('5000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 8000]
team_data = team_data[team_data['MP'] <= 8999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('5000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/nash_peaks.csv', encoding='utf8', index_col=False)
team_data['OffRtg+'] = team_data['OffRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 9000]
team_data = team_data[team_data['MP'] <= 9999]

team_data = team_data.sort_values('OffRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('5000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['OffRtg+'])

team_data = team_data[(team_data['OffRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

In [ ]:
# @title 1000-5000 MP DefRtg+ Peaks
team_data = pd.read_csv('/content/all_peaks_and_single_playoffs_relative_team_data.csv', encoding='utf8', index_col=False)
team_data['DefRtg+'] = team_data['DefRtg+'].round(1)

team = "CHI"

team_data = team_data[team_data['MP'] >= 1000]
team_data = team_data[team_data['MP'] <= 1999]

team_data = team_data.sort_values('DefRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('1000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['DefRtg+'])

team_data = team_data[(team_data['DefRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/all_peaks_and_single_playoffs_relative_team_data.csv', encoding='utf8', index_col=False)
team_data['DefRtg+'] = team_data['DefRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 2000]
team_data = team_data[team_data['MP'] <= 2999]

team_data = team_data.sort_values('DefRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('2000_MP_Team_Peaks.csv',index=False)



only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['DefRtg+'])

team_data = team_data[(team_data['DefRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/all_peaks_and_single_playoffs_relative_team_data.csv', encoding='utf8', index_col=False)
team_data['DefRtg+'] = team_data['DefRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 3000]
team_data = team_data[team_data['MP'] <= 3999]

team_data = team_data.sort_values('DefRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('3000_MP_Team_Peaks.csv',index=False)



only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['DefRtg+'])

team_data = team_data[(team_data['DefRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/all_peaks_and_single_playoffs_relative_team_data.csv', encoding='utf8', index_col=False)
team_data['DefRtg+'] = team_data['DefRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 4000]
team_data = team_data[team_data['MP'] <= 4999]

team_data = team_data.sort_values('DefRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('4000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['DefRtg+'])

team_data = team_data[(team_data['DefRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

team_data = pd.read_csv('/content/all_peaks_and_single_playoffs_relative_team_data.csv', encoding='utf8', index_col=False)
team_data['DefRtg+'] = team_data['DefRtg+'].round(1)

team_data = team_data[team_data['MP'] >= 5000]
team_data = team_data[team_data['MP'] <= 5999]

team_data = team_data.sort_values('DefRtg+', ascending=False)
team_data = team_data.reset_index(drop=True)
team_data.to_csv('5000_MP_Team_Peaks.csv',index=False)


only_team = team_data[(team_data['Team'] == team)]
only_team = only_team.drop_duplicates('Team', keep='first')
print(only_team)

top_off = float(only_team['DefRtg+'])

team_data = team_data[(team_data['DefRtg+'] == top_off)]

print(team_data)

print("\n\n\n\n")

**TEAM PEAK FUNCTIONS**

In [ ]:
#@title X Year Peaks Functions (Team)

# returns if all values in a column are unique.


# def x_yearpeak(source_df, valuestring, number_of_seasons_peak):
# returns a dataframe containing x year stretches of 'valuestring' contained within pandas dataframe, 'df'.
# returned dataframe containts x year stretches of 'valuestring' AND the listed years from each x year stretch + minutes played across the 2 seasons.
def x_yearpeak(source_df, valuestring, valuestring_2, valuestring_3, number_of_seasons_peak):
  x_year_peak = pd.DataFrame(columns = ['Years', 'Team', f'{valuestring}', f'{valuestring_2}', f'{valuestring_3}', 'MP'])

  team_finished = []

  for idx, team in source_df.iterrows():

    # if player has not already been through function call
    if team['Team'] not in team_finished:
      single_team_df = source_df[(source_df['Team'] == team['Team'])]
      single_team_df = single_team_df.reset_index(drop=True)
      number_of_total_team_seasons = len(single_team_df.index)

      for season_index in range(0, number_of_total_team_seasons - (number_of_seasons_peak - 1)):

        years = ''
        non_consecutive_years = 0

        stretch_of_peak_seasons = single_team_df.iloc[season_index: season_index+number_of_seasons_peak].copy()
        stretch_of_peak_seasons = stretch_of_peak_seasons.reset_index(drop=True)
        mp = int(stretch_of_peak_seasons['MP'].sum())
        stretch_of_peak_seasons[f'{valuestring}_Portion'] = stretch_of_peak_seasons[f'{valuestring}'] * (stretch_of_peak_seasons['MP'] / mp)
        stretch_of_peak_seasons[f'{valuestring_2}_Portion'] = stretch_of_peak_seasons[f'{valuestring_2}'] * (stretch_of_peak_seasons['MP'] / mp)
        stretch_of_peak_seasons[f'{valuestring_3}_Portion'] = stretch_of_peak_seasons[f'{valuestring_3}'] * (stretch_of_peak_seasons['MP'] / mp)
        valuestring_peak = stretch_of_peak_seasons[f'{valuestring}_Portion'].sum().round(2)
        valuestring_peak_2 = stretch_of_peak_seasons[f'{valuestring_2}_Portion'].sum().round(2)
        valuestring_peak_3 = stretch_of_peak_seasons[f'{valuestring_3}_Portion'].sum().round(2)

        prev_year = stretch_of_peak_seasons.iloc[0]['Year']
        for year in stretch_of_peak_seasons['Year']:
          if year - prev_year != 1 and year - prev_year != 0:
            stretch_of_peak_seasons["Year"] = stretch_of_peak_seasons["Year"].astype(str)
            years = ', '.join(stretch_of_peak_seasons["Year"])
            non_consecutive_years = 1
            break
          prev_year = year
        if non_consecutive_years == 0:
          years = f"{stretch_of_peak_seasons.iloc[0]['Year']} - {stretch_of_peak_seasons.iloc[number_of_seasons_peak-1]['Year']}"

        cols = ['Years', 'Team', f'{valuestring}', f'{valuestring_2}', f'{valuestring_3}', 'MP']
        df_temp = pd.DataFrame([[years, stretch_of_peak_seasons.iloc[0]['Team'], valuestring_peak, valuestring_peak_2, valuestring_peak_3, mp]], columns=cols)
        x_year_peak = pd.concat([x_year_peak, df_temp], ignore_index=False)
        outfile = f"{number_of_seasons_peak}_year_peak_relative_team_data.csv"
        x_year_peak.to_csv(outfile, index=False)
      team_finished.append(stretch_of_peak_seasons.iloc[0]['Team'])

In [ ]:
# @title Run 2-13 Year Team Data Peaks
team_data = pd.read_csv('/content/Nash_Lebron.csv', index_col=False, encoding='utf8')

all_peaks = pd.DataFrame()

years = 13
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = year7.replace(',', '')
    year7 = int(year7)

    year8 = peak_years.split()[7]
    year8 = year8.replace(',', '')
    year8 = int(year8)

    year9 = peak_years.split()[8]
    year9 = year9.replace(',', '')
    year9 = int(year9)

    year10 = peak_years.split()[9]
    year10 = year10.replace(',', '')
    year10 = int(year10)

    year11 = peak_years.split()[10]
    year11 = year11.replace(',', '')
    year11 = int(year11)

    year12 = peak_years.split()[11]
    year12 = year12.replace(',', '')
    year12 = int(year12)

    year13 = peak_years.split()[12]
    year13 = int(year13)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    elif year8 - year7 >= 5:
      continue
    elif year9 - year8 >= 5:
      continue
    elif year10 - year9 >= 5:
      continue
    elif year11 - year10 >= 5:
      continue
    elif year12 - year11 >= 5:
      continue
    elif year13 - year12 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 12
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = year7.replace(',', '')
    year7 = int(year7)

    year8 = peak_years.split()[7]
    year8 = year8.replace(',', '')
    year8 = int(year8)

    year9 = peak_years.split()[8]
    year9 = year9.replace(',', '')
    year9 = int(year9)

    year10 = peak_years.split()[9]
    year10 = year10.replace(',', '')
    year10 = int(year10)

    year11 = peak_years.split()[10]
    year11 = year11.replace(',', '')
    year11 = int(year11)

    year12 = peak_years.split()[11]
    year12 = int(year12)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    elif year8 - year7 >= 5:
      continue
    elif year9 - year8 >= 5:
      continue
    elif year10 - year9 >= 5:
      continue
    elif year11 - year10 >= 5:
      continue
    elif year12 - year11 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 11
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = year7.replace(',', '')
    year7 = int(year7)

    year8 = peak_years.split()[7]
    year8 = year8.replace(',', '')
    year8 = int(year8)

    year9 = peak_years.split()[8]
    year9 = year9.replace(',', '')
    year9 = int(year9)

    year10 = peak_years.split()[9]
    year10 = year10.replace(',', '')
    year10 = int(year10)

    year11 = peak_years.split()[10]
    year11 = int(year11)


    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    elif year8 - year7 >= 5:
      continue
    elif year9 - year8 >= 5:
      continue
    elif year10 - year9 >= 5:
      continue
    elif year11 - year10 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 10
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = year7.replace(',', '')
    year7 = int(year7)

    year8 = peak_years.split()[7]
    year8 = year8.replace(',', '')
    year8 = int(year8)

    year9 = peak_years.split()[8]
    year9 = year9.replace(',', '')
    year9 = int(year9)

    year10 = peak_years.split()[9]
    year10 = int(year10)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    elif year8 - year7 >= 5:
      continue
    elif year9 - year8 >= 5:
      continue
    elif year10 - year9 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 9
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = year7.replace(',', '')
    year7 = int(year7)

    year8 = peak_years.split()[7]
    year8 = year8.replace(',', '')
    year8 = int(year8)

    year9 = peak_years.split()[8]
    year9 = int(year9)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    elif year8 - year7 >= 5:
      continue
    elif year9 - year8 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 8
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = year7.replace(',', '')
    year7 = int(year7)

    year8 = peak_years.split()[7]
    year8 = int(year8)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    elif year8 - year7 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 7
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = year6.replace(',', '')
    year6 = int(year6)

    year7 = peak_years.split()[6]
    year7 = int(year7)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    elif year7 - year6 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 6
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = year5.replace(',', '')
    year5 = int(year5)

    year6 = peak_years.split()[5]
    year6 = int(year6)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    elif year6 - year5 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 5
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = year4.replace(',', '')
    year4 = int(year4)

    year5 = peak_years.split()[4]
    year5 = int(year5)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    elif year5 - year4 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 4
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = year3.replace(',', '')
    year3 = int(year3)

    year4 = peak_years.split()[3]
    year4 = int(year4)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
    elif year4 - year3 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 3
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = year2.replace(',', '')
    year2 = int(year2)

    year3 = peak_years.split()[2]
    year3 = int(year3)

    if year2 - year1 >= 5:
      continue
    elif year3 - year2 >= 5:
      continue
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)

years = 2
stat = 'OffRtg+'
stat_2 = 'DefRtg+'
stat_3 = 'Relative NetRtg'
x_yearpeak(team_data, stat, stat_2, stat_3, years)
import_team_data = pd.read_csv(f"{years}_year_peak_relative_team_data.csv", encoding='utf8')
copy = import_team_data.copy()
new_df = pd.DataFrame()
for idx, row in copy.iterrows():
  peak_years = row['Years']
  dash_substring = '-'
  if dash_substring in peak_years:
    listed_row = [copy.iloc[idx]]
    tmp_df = pd.DataFrame(listed_row)
    new_df = pd.concat([new_df, tmp_df], ignore_index=False)
  else:
    year1 = peak_years.split()[0]
    year1 = year1.replace(',', '')
    year1 = int(year1)

    year2 = peak_years.split()[1]
    year2 = int(year2)

    if year2 - year1 >= 5:
      continue
    else:
      listed_row = [copy.iloc[idx]]
      tmp_df = pd.DataFrame(listed_row)
      new_df = pd.concat([new_df, tmp_df], ignore_index=False)
new_df.to_csv(f"{years}_year_peak_relative_team_data.csv", index=False)
print(new_df)
all_peaks = pd.concat([all_peaks, new_df], ignore_index=False)
all_peaks.to_csv(f"all_peaks_relative_team_data.csv", index=False)